In [4]:
# get max test id so far
import os
test_ids = [int(f.split("_")[1].split(".")[0]) for f in os.listdir("parsed_tests") if f.startswith("test_")]

max_test_id = max(test_ids)
max_test_id

976691

In [6]:
# Krok 1: Instalace potřebných knihoven
# Odkomentujte a spusťte následující řádek, pokud knihovny ještě nemáte nainstalované
# !pip install requests pdfplumber

import requests
import io
import pdfplumber
import re
import json
import os
from pprint import pprint

def analyzuj_test_z_pdf(url: str):
    """
    Stáhne PDF test z dané URL, vyparsuje z něj otázky a odpovědi
    pomocí robustní metody, která zvládá víceřádkové otázky a odstraňuje patičky.
    """
    print(f"Stahuji PDF z adresy: {url}...")

    try:
        # Stažení PDF souboru s hlavičkou pro obejití základní ochrany
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, timeout=30, headers=headers)
        response.raise_for_status()
        print("PDF úspěšně staženo.")

        pdf_content = response.content

        # Extrakce textu z PDF
        print("Parsuji obsah PDF...")
        full_text = ""
        with io.BytesIO(pdf_content) as pdf_stream:
            with pdfplumber.open(pdf_stream) as pdf:
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

        if "Přezkušovací test ULL Pilot" not in full_text:
            print("PDF neobsahuje očekávaný text, zřejmě se nejedná o ULL test nebo stránka neexistuje.")
            return None

        # --- NOVÝ KROK: ČIŠTĚNÍ TEXTU OD PATIČEK ---
        # Odstraníme všechny řádky, které začínají "Tisk:", abychom se zbavili patiček.
        full_text = re.sub(r'^Tisk:.*$', '', full_text, flags=re.MULTILINE)
        # ---------------------------------------------

        print("Extrakce textu dokončena. Zpracovávám otázky...")

        pattern_otazky = re.compile(r'(\d+)\.\s+([\s\S]+?)Počet bodů:\s*(\d+)', re.MULTILINE)
        otazky = []
        matches = pattern_otazky.finditer(full_text)

        for match in matches:
            blok_otazky = match.group(2).strip()
            body_otazky = int(match.group(3))

            prvni_odpoved_match = re.search(r'\n\s*[A-C]\.', blok_otazky)

            if prvni_odpoved_match:
                index_odpovedi = prvni_odpoved_match.start()
                text_otazky_blok = blok_otazky[:index_odpovedi].strip()
                odpovedi_blok = blok_otazky[index_odpovedi:].strip()
            else:
                text_otazky_blok = blok_otazky
                odpovedi_blok = ""

            text_otazky = ' '.join(text_otazky_blok.split())

            moznosti = {}
            spravna_odpoved = None

            # Regex pro nalezení jednotlivých odpovědí v bloku
            # Hledá písmeno, tečku, a pak vše až do dalšího písmene s tečkou na začátku řádku nebo do konce bloku
            pattern_odpovedi = re.compile(r'([A-C])\.\s*([\s\S]*?)(?=\n[A-C]\.|\Z)', re.MULTILINE)
            odpovedi_matches = pattern_odpovedi.finditer(odpovedi_blok)

            for odpoved_match in odpovedi_matches:
                pismeno = odpoved_match.group(1)
                text_odpovedi_surovy = odpoved_match.group(2)

                if '☺' in text_odpovedi_surovy or '☻' in text_odpovedi_surovy:
                    spravna_odpoved = pismeno

                cisty_text = re.sub(r'[x☺☻●]', '', text_odpovedi_surovy).strip()
                moznosti[pismeno] = ' '.join(cisty_text.split())

            otazky.append({
                "text_otazky": text_otazky,
                "moznosti": moznosti,
                "spravna_odpoved": spravna_odpoved,
                "body": body_otazky
            })

        datum_match = re.search(r'Datum\s+([\d.]+)', full_text)
        datum_testu = datum_match.group(1) if datum_match else "N/A"

        print("Zpracování dokončeno.")
        return {
            "datum_testu": datum_testu,
            "prehled_otazek": otazky,
        }

    except requests.exceptions.HTTPError as e:
        print(f"Test na adrese {url} pravděpodobně neexistuje (HTTP chyba: {e.response.status_code}).")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Chyba při stahování PDF: {e}")
        return None
    except Exception as e:
        print(f"Nastala neočekávaná chyba při parsování: {e}")
        return None

# --- Spouštěcí část skriptu (vaše smyčka) ---

if not os.path.exists("parsed_tests"):
    os.makedirs("parsed_tests")
    print("Vytvořena složka 'parsed_tests'")

url_base = "https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id="
start_id = max_test_id
end_id = max_test_id+2000

chyba_count = 0
max_chyb = 30

for i in range(start_id, end_id + 1):
    nazev_souboru = f"test_{i}.json"
    cesta_k_souboru = os.path.join("parsed_tests", nazev_souboru)

    if os.path.exists(cesta_k_souboru):
        print(f"Test s ID {i} již existuje, přeskakuji.")
        chyba_count = 0
        continue

    print(f"\n--- Zpracovávám test s ID {i} ---")
    url_pdf_testu = url_base + str(i)

    zpracovana_data = analyzuj_test_z_pdf(url_pdf_testu)

    if zpracovana_data is None or not zpracovana_data.get("prehled_otazek"):
        print(f"Test s ID {i} se nepodařilo zpracovat nebo je prázdný. Ukládám prázdný soubor.")
        chyba_count += 1
        if chyba_count >= max_chyb:
            print(f"Počet po sobě jdoucích chyb dosáhl {max_chyb}, ukončuji zpracování.")
            break
        continue

    chyba_count = 0

    print(f"Úspěšně zpracováno {len(zpracovana_data['prehled_otazek'])} otázek. Ukládám do souboru {nazev_souboru}.")
    with open(cesta_k_souboru, "w", encoding='utf-8') as f:
        json.dump(zpracovana_data, f, ensure_ascii=False, indent=4)

print("\n--- Zpracování všech ID dokončeno. ---")

Test s ID 976691 již existuje, přeskakuji.

--- Zpracovávám test s ID 976692 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=976692...
PDF úspěšně staženo.
Parsuji obsah PDF...


KeyboardInterrupt: 

In [15]:
# Krok 1: Instalace potřebných knihoven
# Odkomentujte a spusťte následující řádek, pokud knihovny ještě nemáte nainstalované
# !pip install requests pdfplumber tqdm

import requests
import io
import pdfplumber
import re
import json
import os
import time
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ==============================================================================
# ČÁST 1: FUNKCE PRO PARSOVÁNÍ (zůstává téměř stejná)
# ==============================================================================

def analyzuj_test_z_pdf(pdf_content: bytes):
    """
    Analyzuje obsah PDF a extrahuje data testu.
    Tato funkce je volána z workeru. Vrací data nebo None.
    """
    try:
        full_text = ""
        with io.BytesIO(pdf_content) as pdf_stream:
            with pdfplumber.open(pdf_stream) as pdf:
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

        if "Přezkušovací test ULL Pilot" not in full_text:
            return None # Není to platný test

        # Odstranění patiček pro čistší parsování
        full_text = re.sub(r'^Tisk:.*$', '', full_text, flags=re.MULTILINE)

        pattern_otazky = re.compile(r'(\d+)\.\s+([\s\S]+?)Počet bodů:\s*(\d+)', re.MULTILINE)
        otazky = []
        matches = pattern_otazky.finditer(full_text)

        for match in matches:
            blok_otazky = match.group(2).strip()
            body_otazky = int(match.group(3))

            prvni_odpoved_match = re.search(r'\n\s*[A-C]\.', blok_otazky)

            if prvni_odpoved_match:
                index_odpovedi = prvni_odpoved_match.start()
                text_otazky_blok = blok_otazky[:index_odpovedi].strip()
                odpovedi_blok = blok_otazky[index_odpovedi:].strip()
            else:
                text_otazky_blok = blok_otazky
                odpovedi_blok = ""

            text_otazky = ' '.join(text_otazky_blok.split())

            moznosti = {}
            spravna_odpoved = None

            pattern_odpovedi = re.compile(r'([A-C])\.\s*([\s\S]*?)(?=\n[A-C]\.|\Z)', re.MULTILINE)
            odpovedi_matches = pattern_odpovedi.finditer(odpovedi_blok)

            for odpoved_match in odpovedi_matches:
                pismeno = odpoved_match.group(1)
                text_odpovedi_surovy = odpoved_match.group(2)

                if '☺' in text_odpovedi_surovy or '☻' in text_odpovedi_surovy:
                    spravna_odpoved = pismeno

                cisty_text = re.sub(r'[x☺☻●]', '', text_odpovedi_surovy).strip()
                moznosti[pismeno] = ' '.join(cisty_text.split())

            otazky.append({
                "text_otazky": text_otazky,
                "moznosti": moznosti,
                "spravna_odpoved": spravna_odpoved,
                "body": body_otazky
            })

        datum_match = re.search(r'Datum\s+([\d.]+)', full_text)
        datum_testu = datum_match.group(1) if datum_match else "N/A"

        return {
            "datum_testu": datum_testu,
            "prehled_otazek": otazky,
        }
    except Exception:
        # Pokud parsování selže, vrátíme None
        return None

# ==============================================================================
# ČÁST 2: FUNKCE PRO JEDNOHO WORKERA (nová funkce)
# ==============================================================================

def zpracuj_jeden_test(test_id):
    """
    Kompletní logika pro zpracování jednoho ID: kontrola existence, stažení,
    parsování a uložení souboru. Vrací stav zpracování.
    """
    url_base = "https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id="
    vystupni_slozka = "parsed_tests"

    nazev_souboru = f"test_{test_id}.json"
    cesta_k_souboru = os.path.join(vystupni_slozka, nazev_souboru)

    if os.path.exists(cesta_k_souboru):
        return "skipped" # Přeskočeno

    url_pdf = url_base + str(test_id)

    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url_pdf, timeout=15, headers=headers)

        # Pokud stránka neexistuje (404) nebo jiná chyba, rovnou selžeme
        if response.status_code != 200:
            with open(cesta_k_souboru, "w") as f: json.dump({}, f)
            return "failed"

        # Zpracování PDF obsahu
        zpracovana_data = analyzuj_test_z_pdf(response.content)

        # Uložení výsledku (i neúspěšného parsování)
        if zpracovana_data and zpracovana_data.get("prehled_otazek"):
            with open(cesta_k_souboru, "w", encoding='utf-8') as f:
                json.dump(zpracovana_data, f, ensure_ascii=False, indent=4)
            return "success"
        else:
            with open(cesta_k_souboru, "w") as f: json.dump({}, f)
            return "failed"

    except requests.exceptions.RequestException:
        # Chyba sítě/timeoutu
        with open(cesta_k_souboru, "w") as f: json.dump({}, f)
        return "failed"

# ==============================================================================
# ČÁST 3: HLAVNÍ SPOUŠTĚCÍ BLOK
# ==============================================================================

if __name__ == "__main__":
    # --- Nastavení ---
    start_id = max_test_id
    end_id = max_test_id+1000
    vystupni_slozka = "parsed_tests"
    # Použijeme všechna dostupná jádra CPU, ale maximálně např. 16, abychom nezahltili server
    pocet_workeru = 4

    # Vytvoření složky pro výsledky, pokud neexistuje
    if not os.path.exists(vystupni_slozka):
        os.makedirs(vystupni_slozka)
        print(f"Vytvořena složka '{vystupni_slozka}'")

    # Seznam všech ID, které chceme zpracovat
    vsechna_id = range(start_id, end_id + 1)

    print(f"Spouštím zpracování {len(vsechna_id)} testů na {pocet_workeru} jádrech CPU.")

    # Vytvoření poolu workerů
    with Pool(processes=pocet_workeru) as pool:
        # Použijeme pool.imap_unordered pro efektivní zpracování s progress barem
        # imap_unordered vrací výsledky, jakmile jsou hotové, což je ideální pro TQDM
        results = list(tqdm(pool.imap_unordered(zpracuj_jeden_test, vsechna_id), total=len(vsechna_id)))

    # Vyhodnocení výsledků
    uspesne = results.count("success")
    preskoceno = results.count("skipped")
    neúspěšné = results.count("failed")

    print("\n--- Zpracování dokončeno ---")
    print(f"Celkem úspěšně zpracováno: {uspesne}")
    print(f"Celkem přeskočeno (již existovalo): {preskoceno}")
    print(f"Celkem neúspěšně (chyba nebo neexistuje): {neúspěšné}")

Spouštím zpracování 1001 testů na 4 jádrech CPU.


100%|██████████| 1001/1001 [01:15<00:00, 13.30it/s]


--- Zpracování dokončeno ---
Celkem úspěšně zpracováno: 516
Celkem přeskočeno (již existovalo): 73
Celkem neúspěšně (chyba nebo neexistuje): 412


In [2]:
import os
import json
import duckdb
from datetime import datetime

# --- Configuration ---
# The name of the DuckDB database file to create/use.
DB_FILE = "tests.duckdb"
# The source directory containing your JSON files.
SOURCE_DIR = "parsed_tests"

# --- Main Import Logic ---

# Check if the source directory exists
if not os.path.isdir(SOURCE_DIR):
    print(f"Error: Source directory '{SOURCE_DIR}' not found.")
    print("Please make sure your JSON files are in that folder.")
else:
    # Connect to the DuckDB database file (it will be created if it doesn't exist)
    with duckdb.connect(database=DB_FILE, read_only=False) as conn:

        # 1. Create the 'questions' table if it doesn't already exist.
        # This schema must match the one in your main script.
        conn.execute("""
        CREATE TABLE IF NOT EXISTS questions (
            test_id UINTEGER,
            test_date DATE,
            question_text VARCHAR,
            option_a VARCHAR,
            option_b VARCHAR,
            option_c VARCHAR,
            correct_option VARCHAR(1),
            points UTINYINT
        );
        """)
        print(f"Database '{DB_FILE}' is ready. Starting import from '{SOURCE_DIR}'...")

        processed_files = 0
        total_questions_inserted = 0

        # 2. Loop through all files in the source directory
        for filename in os.listdir(SOURCE_DIR):
            if filename.endswith('.json'):
                file_path = os.path.join(SOURCE_DIR, filename)

                try:
                    # Extract the test ID from the filename (e.g., 'test_12345.json' -> 12345)
                    test_id = int(filename.split('_')[1].split('.')[0])

                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                    # Skip empty or malformed files
                    if not data or 'prehled_otazek' not in data:
                        print(f"  - Skipping empty or invalid file: {filename}")
                        continue

                    # Parse test date, handling potential errors
                    test_date_str = data.get('datum_testu')
                    test_date = None
                    if test_date_str:
                        try:
                            test_date = datetime.strptime(test_date_str, "%d.%m.%Y").date()
                        except ValueError:
                            print(f"  - Warning: Invalid date format in {filename}. Storing as NULL.")

                    # 3. Loop through each question in the JSON and insert it into the DB
                    for question in data.get('prehled_otazek', []):
                        conn.execute(
                            "INSERT INTO questions VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                            [
                                test_id,
                                test_date,
                                question.get('text_otazky'),
                                question.get('moznosti', {}).get('A'),
                                question.get('moznosti', {}).get('B'),
                                question.get('moznosti', {}).get('C'),
                                question.get('spravna_odpoved'),
                                question.get('body')
                            ]
                        )
                        total_questions_inserted += 1

                    processed_files += 1

                except (json.JSONDecodeError, IndexError, KeyError) as e:
                    print(f"  - Error processing file {filename}: {e}. Skipping.")

        print("\n--- Import Finished ---")
        print(f"Processed {processed_files} JSON files.")
        print(f"Inserted a total of {total_questions_inserted} questions into the database.")

        # 4. (Optional) Verify the import by counting the rows in the table
        result = conn.execute("SELECT COUNT(*) FROM questions").fetchone()
        if result:
            print(f"Verification: The 'questions' table now contains {result[0]} rows.")

Database 'tests.duckdb' is ready. Starting import from 'parsed_tests'...
  - Skipping empty or invalid file: test_950002.json
  - Skipping empty or invalid file: test_950005.json
  - Skipping empty or invalid file: test_950010.json
  - Skipping empty or invalid file: test_950011.json
  - Skipping empty or invalid file: test_950013.json
  - Skipping empty or invalid file: test_950014.json
  - Skipping empty or invalid file: test_950016.json
  - Skipping empty or invalid file: test_950017.json
  - Skipping empty or invalid file: test_950018.json
  - Skipping empty or invalid file: test_950024.json
  - Skipping empty or invalid file: test_950026.json
  - Skipping empty or invalid file: test_950027.json
  - Skipping empty or invalid file: test_950028.json
  - Skipping empty or invalid file: test_950030.json
  - Skipping empty or invalid file: test_950034.json
  - Skipping empty or invalid file: test_950035.json
  - Skipping empty or invalid file: test_950039.json
  - Skipping empty or inva

KeyboardInterrupt: 